In [7]:
import datetime
#import logging as log
import os
#import pickle
import random
from collections import deque

import numpy as np
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from retro_contest.local import make
import gym_remote.exceptions as gre

C:\Users\cenarugu\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
# Import the gym module
import gym

# Create a breakout environment
env = gym.make('CartPole-v0')
# Reset it, returns the starting frame
frame = env.reset()
# Render
env.render()

is_done = False
while not is_done:
  # Perform a random action, returns the new frame, reward and whether the game is over
  frame, reward, is_done, _ = env.step(env.action_space.sample())
  # Render
  env.render()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Create model

In [8]:
env = make(game='SonicTheHedgehog-Genesis', state='LabyrinthZone.Act1')

In [9]:
state_size = env.observation_space.shape
action_size = env.action_space.n
learning_rate = 0.001
gamma = 0.90
exploration_rate = 1.0
exploration_min = 0.01
exploration_decay = 0.995
memory = deque(maxlen=10000)
randBinList = lambda n: [random.randint(0, 1) for b in range(1, n + 1)]


In [10]:
def create_model(): 
    model = Sequential()
    model.add(Conv2D(32, 8, 8, subsample=(4, 4), input_shape=(state_size)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, 4, 4, subsample=(2, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(units=64))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=32))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(output_dim=action_size, activation='softmax'))

    adam = Adam(lr=learning_rate)
    model.compile(loss='mae', optimizer=adam)

In [1]:
def act(self, state):
        #Buttons: ['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']
        if np.random.random() <= self.exploration_rate:
            return self.randBinList(self.action_size)
        q_values = self.model.predict(state)
        return q_values[0]

In [2]:
def get_sample_batch(self, sample_batch_size):
        return random.sample(self.memory, sample_batch_size)

In [3]:
def replay(self, sample_batch_size):

        if len(self.memory) < sample_batch_size:
            return

        sample_batch = get_sample_batch(sample_batch_size)
        for state, action, reward, next_state, done in sample_batch:
            if done:
                Q_values = np.array([reward])
            else:
                Q_values = reward + self.gamma * np.max(model.predict(next_state), axis=1)

            state = np.reshape(state, (1,) + self.state_size)
            model.train_on_batch(state, reward_value)(state, action*Q_values[:, None])
        if exploration_rate > exploration_min:
            exploration_rate *= exploration_decay

In [11]:
episodes = 1000

for index_episode in range(episodes):
                state = env.reset()
                state = np.reshape(state, (1,)+state_size)

                done = False
                tot_reward = 0

                while not done:
                    env.render()
                    action = self.agent.act(state)
                    next_state, reward, done, _ = env.step(action)
                    next_state = np.reshape(next_state, (1,)+state_size)
                    self.agent.remember(state, action, reward, next_state, done)
                    state = next_state
                    tot_reward += reward
                #if (index_episode % self.checkpoint == 0 and index_episode is not 0):
                #    self.agent.save_model_and_memory(WEIGHT_BACKUP_NAME, self.backup_folder)
                print("Episode {}# Score: {}".format(index_episode, tot_reward))
                agent.replay(self.sample_batch_size)
                #self.env.reset()

NotImplementedError: abstract